In [1]:
import findspark
findspark.init()
findspark.find()

'/Users/vikashgarg/opt/anaconda3/lib/python3.9/site-packages/pyspark'

In [2]:
!pip install great_expectations==0.15.31

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark=SparkSession.builder.appName('ge').getOrCreate()

22/12/12 13:33:30 WARN Utils: Your hostname, Vikashs-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.8 instead (on interface en0)
22/12/12 13:33:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/12 13:33:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
spark

In [8]:
from pyspark.sql.functions import *

In [9]:
df=spark.read.format('csv').options(header = "true",inferSchema = "true", sep = ",", quote = "\"", escape = "\"").load('Dataset.csv')

In [10]:
df.show()

+-----------------+----------------+---+------+-------------------+
|               ID|            name|age|gender|              email|
+-----------------+----------------+---+------+-------------------+
|                1|     Peter Poole| 30|  Male|      po@zalowpo.az|
|                2|   Herbert Payne| 46|Female|      zos@toilet.bn|
|                3|      Callie Roy| 57|  Male|kapkapor@reckeho.ye|
|                4|  Isabella Wolfe| 61|Female| atuigome@asedas.io|
|                5|     Sara Knight| 53|Female| dudnukob@ucboki.tg|
|                6| Raymond Herrera| 23|  Male|       tolpij@ni.lu|
|                7|  Jeffery Brooks| 65|  Male|   anirus@ajjuhu.mw|
|                8|  Lillie Barnett| 55|  Male|      besi@vodje.hr|
|                9|    Emma Alvarez| 21|  Male|          ze@ebo.jm|
|               10|  Estella Rhodes| 27|  Male|         ez@jewe.si|
|               11|     Eugene West| 18|Female| hohobum@janucke.vn|
|               12|     Ollie Blake| 57|  Male| 

In [11]:
import datetime
 

from ruamel import yaml
from great_expectations import *
from great_expectations.core.batch import RuntimeBatchRequest
from great_expectations.data_context import BaseDataContext
from great_expectations.data_context.types.base import (
    DataContextConfig,
    FilesystemStoreBackendDefaults,
)
 
root_directory = "/Users/vikashgarg/Downloads/ge"

In [12]:

#create Data Context
data_context_config = DataContextConfig(
    store_backend_defaults=FilesystemStoreBackendDefaults(
        root_directory=root_directory
    ),
)
context = BaseDataContext(project_config=data_context_config)

In [13]:

my_spark_datasource_config = {
    "name": "insert_your_datasource_name_here",
    "class_name": "Datasource",
    "execution_engine": {"class_name": "SparkDFExecutionEngine"},
    "data_connectors": {
        "insert_your_data_connector_name_here": {
            "module_name": "great_expectations.datasource.data_connector",
            "class_name": "RuntimeDataConnector",
            "batch_identifiers": [
                "some_key_maybe_pipeline_stage",
                "some_other_key_maybe_run_id",
            ],
        }
    },
}
context.test_yaml_config(yaml.dump(my_spark_datasource_config))
 
context.add_datasource(**my_spark_datasource_config)
 
batch_request = RuntimeBatchRequest(
    datasource_name="insert_your_datasource_name_here",
    data_connector_name="insert_your_data_connector_name_here",
    data_asset_name="<YOUR_MEANGINGFUL_NAME>",  # This can be anything that identifies this data_asset for you
    batch_identifiers={
        "some_key_maybe_pipeline_stage": "prod",
        "some_other_key_maybe_run_id": f"my_run_name_{datetime.date.today().strftime('%Y%m%d')}",
    },
    runtime_parameters={"batch_data": df},  
)
expectation_suite_name = "insert_your_expectation_suite_name_here"
context.create_expectation_suite(
    expectation_suite_name=expectation_suite_name, overwrite_existing=True
)
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=expectation_suite_name,
)

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
22/12/12 13:20:31 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
	Successfully instantiated Datasource


ExecutionEngine class name: SparkDFExecutionEngine
Data Connectors:
	insert_your_data_connector_name_here:RuntimeDataConnector

	Available data_asset_names (0 of 0):
		Note : RuntimeDataConnector will not have data_asset_names until they are passed in through RuntimeBatchRequest

	Unmatched data_references (0 of 0): []



In [14]:
# Databricks notebook source
import json
import re  # regular expressions

# !!! This giant block of imports should be something simpler, such as:
from great_expectations import *
from great_expectations.execution_engine import (
    PandasExecutionEngine,
    SparkDFExecutionEngine,
    SqlAlchemyExecutionEngine,
)
from great_expectations.expectations.expectation import (
    ColumnMapExpectation,
    Expectation,
    ExpectationConfiguration,
)
from great_expectations.expectations.metrics import (
    ColumnMapMetricProvider,
    column_condition_partial,
)
from great_expectations.expectations.registry import (
    _registered_expectations,
    _registered_metrics,
    _registered_renderers,
)
from great_expectations.expectations.util import render_evaluation_parameter_string
from great_expectations.render.renderer.renderer import renderer
from great_expectations.render.types import RenderedStringTemplateContent
from great_expectations.render.util import num_to_str, substitute_none_for_missing
from great_expectations.validator.validator import Validator

EMAIL_REGEX = r"[a-z0-9]+[\._]?[a-z0-9]+[@]\w+[.]\w{2,7}$"


class ColumnValuesContainValidEmail(ColumnMapMetricProvider):
    # This is the id string that will be used to reference your metric.
    condition_metric_name = "column_values.valid_email"
    condition_value_keys = ()

    # This method defines the business logic for evaluating your metric when using a PandasExecutionEngine
    @column_condition_partial(engine=PandasExecutionEngine)
    def _pandas(cls, column, **kwargs):
        def matches_email_regex(x):
            if re.match(EMAIL_REGEX, str(x)):
                return True
            return False

        return column.apply(lambda x: matches_email_regex(x) if x else False)

    # This method defines the business logic for evaluating your metric when using a SqlAlchemyExecutionEngine
    #     @column_condition_partial(engine=SqlAlchemyExecutionEngine)
    #     def _sqlalchemy(cls, column, _dialect, **kwargs):
    #         return column.in_([3])

    # This method defines the business logic for evaluating your metric when using a SparkDFExecutionEngine
    @column_condition_partial(engine=SparkDFExecutionEngine)
    def _spark(cls, column, **kwargs):
        return column.rlike(EMAIL_REGEX)


# This class defines the Expectation itself
# The main business logic for calculation lives here.
class ExpectColumnValuesToContainValidEmail(ColumnMapExpectation):
    # These examples will be shown in the public gallery, and also executed as unit tests for your Expectation
    examples = [
        {
            "data": {
                "fail_case_1": ["a123@something", "a123@something.", "a123."],
                "fail_case_2": ["aaaa.a123.co", "aaaa.a123.", "aaaa.a123.com"],
                "fail_case_3": ["aaaa@a123.e", "aaaa@a123.a", "aaaa@a123.d"],
                "fail_case_4": ["@a123.com", "@a123.io", "@a123.eu"],
                "pass_case_1": [
                    "a123@something.com",
                    "vinod.km@something.au",
                    "this@better.work",
                ],
                "pass_case_2": [
                    "example@website.dom",
                    "ex.ample@example.ex",
                    "great@expectations.email",
                ],
                "valid_emails": [
                    "Janedoe@company.org",
                    "someone123@stuff.net",
                    "mycompany@mycompany.com",
                ],
                "bad_emails": ["Hello, world!", "Sophia", "this should fail"],
            },
            "tests": [
                {
                    "title": "negative_test_for_no_domain_name",
                    "exact_match_out": False,
                    "include_in_gallery": True,
                    "in": {"column": "fail_case_1"},
                    "out": {
                        "success": False,
                        "unexpected_index_list": [0, 1, 2],
                        "unexpected_list": [
                            "a123@something",
                            "a123@something.",
                            "a123.",
                        ],
                    },
                },
                {
                    "title": "negative_test_for_no_at_symbol",
                    "exact_match_out": False,
                    "include_in_gallery": True,
                    "in": {"column": "fail_case_2"},
                    "out": {
                        "success": False,
                        "unexpected_index_list": [0, 1, 2],
                        "unexpected_list": [
                            "aaaa.a123.co",
                            "aaaa.a123.",
                            "aaaa.a123.com",
                        ],
                    },
                },
                {
                    "title": "negative_test_for_ending_with_one_character",
                    "exact_match_out": False,
                    "include_in_gallery": True,
                    "in": {"column": "fail_case_3"},
                    "out": {
                        "success": False,
                        "unexpected_index_list": [0, 1, 2],
                        "unexpected_list": [
                            "aaaa@a123.e",
                            "aaaa@a123.a",
                            "aaaa@a123.d",
                        ],
                    },
                },
                {
                    "title": "negative_test_for_emails_with_no_leading_string",
                    "exact_match_out": False,
                    "include_in_gallery": True,
                    "in": {"column": "fail_case_4"},
                    "out": {
                        "success": False,
                        "unexpected_index_list": [0, 1, 2],
                        "unexpected_list": [
                            "aaaa@a123.e",
                            "aaaa@a123.a",
                            "aaaa@a123.d",
                        ],
                    },
                },
                {
                    "title": "pass_test",
                    "exact_match_out": False,
                    "include_in_gallery": True,
                    "in": {"column": "pass_case_1"},
                    "out": {
                        "success": True,
                        "unexpected_index_list": [],
                        "unexpected_list": [],
                    },
                },
                {
                    "title": "pass_test",
                    "exact_match_out": False,
                    "include_in_gallery": True,
                    "in": {"column": "pass_case_2"},
                    "out": {
                        "success": True,
                        "unexpected_index_list": [],
                        "unexpected_list": [],
                    },
                },
                {
                    "title": "valid_emails",
                    "exact_match_out": False,
                    "include_in_gallery": True,
                    "in": {"column": "valid_emails"},
                    "out": {
                        "success": True,
                        "unexpected_index_list": [],
                        "unexpected_list": [],
                    },
                },
                {
                    "title": "invalid_emails",
                    "exact_match_out": False,
                    "include_in_gallery": True,
                    "in": {"column": "bad_emails"},
                    "out": {
                        "success": False,
                        "unexpected_index_list": [0, 1, 2],
                        "unexpected_list": [
                            "Hello, world!",
                            "Sophia",
                            "this should fail",
                        ],
                    },
                },
            ],
        }
    ]

    # This dictionary contains metadata for display in the public gallery
    library_metadata = {
        "maturity": "experimental",
        "tags": ["experimental", "column map expectation"],
        "contributors": [  # Github
            "@aworld1",
            "@enagola",
            "@spencerhardwick",
            "@vinodkri1",
            "@degulati",
            "@ljohnston931",
            "@rexboyce",
            "@lodeous",
            "@sophiarawlings",
            "@vtdangg",
        ],
    }

    # This is the id string of the Metric used by this Expectation.
    # For most Expectations, it will be the same as the `condition_metric_name` defined in your Metric class above.
    map_metric = "column_values.valid_email"

    # This is a list of parameter names that can affect whether the Expectation evaluates to True or False
    # Please see {some doc} for more information about domain and success keys, and other arguments to Expectations
    success_keys = ()

    # This dictionary contains default values for any parameters that should have default values
    default_kwarg_values = {}

    # This method defines a question Renderer
    # For more info on Renderers, see {some doc}
    # !!! This example renderer should render RenderedStringTemplateContent, not just a string


#     @classmethod
#     @renderer(renderer_type="renderer.question")
#     def _question_renderer(
#         cls, configuration, result=None, language=None, runtime_configuration=None
#     ):
#         column = configuration.kwargs.get("column")
#         mostly = configuration.kwargs.get("mostly")

#         return f'Do at least {mostly * 100}% of values in column "{column}" equal 3?'

# This method defines an answer Renderer
# !!! This example renderer should render RenderedStringTemplateContent, not just a string
#     @classmethod
#     @renderer(renderer_type="renderer.answer")
#     def _answer_renderer(
#         cls, configuration=None, result=None, language=None, runtime_configuration=None
#     ):
#         column = result.expectation_config.kwargs.get("column")
#         mostly = result.expectation_config.kwargs.get("mostly")
#         regex = result.expectation_config.kwargs.get("regex")
#         if result.success:
#             return f'At least {mostly * 100}% of values in column "{column}" equal 3.'
#         else:
#             return f'Less than {mostly * 100}% of values in column "{column}" equal 3.'

# This method defines a prescriptive Renderer
#     @classmethod
#     @renderer(renderer_type="renderer.prescriptive")
#     @render_evaluation_parameter_string
#     def _prescriptive_renderer(
#         cls,
#         configuration=None,
#         result=None,
#         language=None,
#         runtime_configuration=None,
#         **kwargs,
#     ):
# !!! This example renderer should be shorter
#         runtime_configuration = runtime_configuration or {}
#         include_column_name = runtime_configuration.get("include_column_name", True)
#         include_column_name = (
#             include_column_name if include_column_name is not None else True
#         )
#         styling = runtime_configuration.get("styling")
#         params = substitute_none_for_missing(
#             configuration.kwargs,
#             ["column", "regex", "mostly", "row_condition", "condition_parser"],
#         )

#         template_str = "values must be equal to 3"
#         if params["mostly"] is not None:
#             params["mostly_pct"] = num_to_str(
#                 params["mostly"] * 100, precision=15, no_scientific=True
#             )
#             # params["mostly_pct"] = "{:.14f}".format(params["mostly"]*100).rstrip("0").rstrip(".")
#             template_str += ", at least $mostly_pct % of the time."
#         else:
#             template_str += "."

#         if include_column_name:
#             template_str = "$column " + template_str

#         if params["row_condition"] is not None:
#             (
#                 conditional_template_str,
#                 conditional_params,
#             ) = parse_row_condition_string_pandas_engine(params["row_condition"])
#             template_str = conditional_template_str + ", then " + template_str
#             params.update(conditional_params)

#         return [
#             RenderedStringTemplateContent(
#                 **{
#                     "content_block_type": "string_template",
#                     "string_template": {
#                         "template": template_str,
#                         "params": params,
#                         "styling": styling,
#                     },
#                 }
#             )
#         ]

if __name__ == "__main__":
    ExpectColumnValuesToContainValidEmail().print_diagnostic_checklist()

# COMMAND ----------




22/12/12 13:20:40 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/9 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/11 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/11 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/11 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/11 [00:00<?, ?it/s]

3 5 3 False
Completeness checklist for ExpectColumnValuesToContainValidEmail (EXPERIMENTAL):
 ✔ Has a valid library_metadata object
   Has a docstring, including a one-line short description
   Has at least one positive and negative example case, and all test cases pass
   Has core logic and passes tests on at least one Execution Engine
   Passes all linting checks
      inspect.getfile(impl) raised a TypeError (impl is a built-in class)
   Has basic input validation and type checking
      No validate_configuration method defined on subclass
 ✔ Has both statement Renderers: prescriptive and diagnostic
   Has core logic that passes tests for all applicable Execution Engines and SQL dialects
      Only 6 / 8 tests for pandas are passing
        - Failing: negative_test_for_emails_with_no_leading_string, valid_emails
      Only 7 / 8 tests for spark are passing
        - Failing: negative_test_for_emails_with_no_leading_string
   Has a full suite of tests, as determined by a code owner
 

In [15]:
validator.expect_column_values_to_contain_valid_email(column="email")

AttributeError: module 'great_expectations.validator' has no attribute 'expect_column_values_to_contain_valid_email'

In [33]:
validator.

AttributeError: module 'great_expectations.validator' has no attribute 'expect_column_values_to_be_unique'

In [17]:
import great_expectations as ge
import os
from great_expectations.profile.basic_dataset_profiler import BasicDatasetProfiler
from great_expectations.profile.user_configurable_profiler import UserConfigurableProfiler
from great_expectations.dataset.sparkdf_dataset import SparkDFDataset
from great_expectations.render.renderer import *
from great_expectations.render.view import DefaultJinjaPageView
from great_expectations.data_context.util import *
from IPython.display import display
from IPython.core.display import HTML



In [18]:
# read the df

# gdf = ge.from_pandas(df)
gdf = ge.dataset.SparkDFDataset(df)
# gdf = SparkDFDataset(df)
# expectation_suite, validation_result = BasicDatasetProfiler.profile(gdf,profiler_configuration=scaffold_config)
# document_model = ProfilingResultsPageRenderer().render(validation_result)
# os.makedirs(os.path.dirname(profiling_html_filepath), exist_ok=True)
# with open(profiling_html_filepath, 'w') as writer:
#     writer.write(DefaultJinjaPageView.render(document_model))
# display(HTML(DefaultJinjaPageView().render(document_model)))




Py4JJavaError: An error occurred while calling o33.persist.
: java.lang.IllegalStateException: Cannot call methods on a stopped SparkContext.
This stopped SparkContext was created at:

org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServerConnection.waitForCommands(ClientServerConnection.java:182)
py4j.ClientServerCo

In [43]:
gdf.expect_column_values_to_be_unique(column="ID")

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/vikashgarg/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/m7/6z2yy72x74j_x3643mybw9s80000gn/T/ipykernel_1197/3584603313.py", line 1, in <cell line: 1>
    gdf.expect_column_values_to_be_unique(column="ID")
  File "/Users/vikashgarg/opt/anaconda3/lib/python3.9/site-packages/great_expectations/data_asset/util.py", line 76, in f
    return self.mthd(obj, *args, **kwargs)
  File "/Users/vikashgarg/opt/anaconda3/lib/python3.9/site-packages/great_expectations/data_asset/data_asset.py", line 289, in wrapper
  File "/Users/vikashgarg/opt/anaconda3/lib/python3.9/site-packages/great_expectations/data_asset/data_asset.py", line 276, in wrapper
    if isinstance(return_obj, dict):
  File "/Users/vikashgarg/opt/anaconda3/lib/python3.9/site-packages/great_expectations/dataset/sparkdf_dataset.py", line 99, in inner_wrapper
  

22/12/10 06:54:42 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 910944 ms exceeds timeout 120000 ms
22/12/10 06:54:42 WARN SparkContext: Killing executors is not supported by current scheduler.
